## Quantum State Classifier

In [14]:
import qutip
import numpy as np
from qutip import *

In [12]:
from qutip import rand_dm, entropy_vn, ptrace, sigmax, sigmay, sigmaz, tensor, expect
import numpy as np

state = rand_dm(4)
state.dims = [[2, 2], [2, 2]]

matrix_features = np.concatenate([state.full().real.flatten(), state.full().imag.flatten()])

purity = state.purity() 
entropy = entropy_vn(state) 
pauli_ops = [tensor(sigmax(), sigmax()), tensor(sigmay(), sigmay()), tensor(sigmaz(), sigmaz())]
pauli_expect = [expect(op, state) for op in pauli_ops] 
rho_A = ptrace(state, 0)
entropy_A = entropy_vn(rho_A)
rho_B = ptrace(state, 1)
entropy_B = entropy_vn(rho_B)  
eigenvalues = np.linalg.eigvals(state.full()).real  

features = np.concatenate([matrix_features, [purity, entropy, entropy_A, entropy_B], pauli_expect, eigenvalues])

features_list = []
labels = []
for _ in range(1000):
    state = rand_dm(4)
    state.dims = [[2,2], [2,2]]
    matrix_features = np.concatenate([state.full().real.flatten(), state.full().imag.flatten()])
    purity = state.purity()
    entropy = entropy_vn(state)
    pauli_expect = [expect(op, state) for op in pauli_ops]
    rho_A = ptrace(state, 0)
    entropy_A = entropy_vn(rho_A)
    rho_B = ptrace(state, 1)
    entropy_B = entropy_vn(rho_B)
    eigenvalues = np.linalg.eigvals(state.full()).real
    features = np.concatenate([matrix_features, [purity, entropy, entropy_A, entropy_B], pauli_expect, eigenvalues])
    features_list.append(features)
    labels.append(1 if concurrence(state) > 0 else 0)
features = np.array(features_list)
labels = np.array(labels)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
model = LogisticRegression().fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(accuracy)

0.765
